In [ ]:
import pandas as pd
import numpy as np
import argparse
import glob
import joblib
# import os

# from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold, train_test_split

CALC_PATHS = '/home/nfs/jludwiczak/af2_cc/af2_multimer/calc'


def get_x(id_: int, rank: int, model: str = "af2", 
          use_pairwise: bool = True):

    single_repr_fns = sorted(glob.glob(f"{CALC_PATHS}/{id_}/*_single_repr_rank_00*"))
    pair_repr_fns = sorted(glob.glob(f"{CALC_PATHS}{id_}/*_pair_repr_rank_00*"))

    mat = np.load(single_repr_fns[rank]).mean(axis=0)
    if use_pairwise:
        mat = np.hstack((mat, np.load(pair_repr_fns[rank]).mean(axis=0).mean(axis=0)))
    return mat

def get_af2_emb(id_: int, model_id: int, use_pairwise: bool):

    single_repr_fns = sorted(glob.glob(f"{CALC_PATHS}/{id_}/*_single_repr_rank_*_model_{model_id+1}_*"))
    pair_repr_fns = sorted(glob.glob(f"{CALC_PATHS}/{id_}/*_pair_repr_rank_*_model_{model_id+1}_*"))


    mat = np.load(single_repr_fns[0]).mean(axis=0)

    if use_pairwise:
        mat = np.hstack((mat, np.load(pair_repr_fns[0]).mean(axis=0).mean(axis=0)))

    return mat



In [ ]:
df = pd.read_csv("tests/set4_homooligomers.csv", sep="\t")
df = df.drop_duplicates(subset="full_sequence", keep="first")
df.parallel.unique()

In [ ]:
from sklearn.model_selection import KFold, train_test_split

from itertools import product

def train(c=10, balanced=0, dual=1, ensemble_size=1, use_pairwise=True, use_scaler=True):

    df = pd.read_csv("tests/set4_homooligomers.csv", sep="\t")
    df = df.drop_duplicates(subset="full_sequence", keep="first")
    results = np.zeros((ensemble_size, 5, len(df), 2))
    model = {}
    probabilities = []
    for j in range(0, ensemble_size):
        for i in range(0, 5): # 5 since we have 5 AF2 models

            X = np.asarray([get_af2_emb(id_, model_id=i, use_pairwise=use_pairwise) for id_ in df.index])
            y = df['parallel'].values
            cv = KFold(n_splits=5, shuffle=True)

            for k, (tr_idx, te_idx) in enumerate(cv.split(X, y)):

                X_tr, X_te = X[tr_idx], X[te_idx]
                y_tr, y_te = y[tr_idx], y[te_idx]

                if use_scaler == 1:
                    sc = StandardScaler()
                    X_tr = sc.fit_transform(X_tr)
                    X_te = sc.transform(X_te)
                    model[f"scaler_{j}_{i}_{k}"] = sc
                clf = LogisticRegression(C=c, max_iter=1000, solver='liblinear',
                                         dual = False if dual == 0 else True, 
                                         class_weight = 'balanced' if balanced == 1 else None) 
                clf.fit(X_tr, y_tr)
                results[j, i, te_idx] = clf.predict_proba(X_te)
                model[f"clf_{j}_{i}_{k}"] = clf

    y_pred_bin = results.mean(axis=0).mean(axis=0).argmax(axis=1)
    results_ = {}
    results_["accuracy"] = accuracy_score(y, y_pred_bin)
    results_["f1"] = f1_score(y, y_pred_bin, average='macro')

    df["y_pred"] = y_pred_bin
    # df["prob_dimer"] = results.mean(axis=0).mean(axis=0)[:, 0]
    # df["prob_trimer"] = results.mean(axis=0).mean(axis=0)[:, 1]
    # df["prob_tetramer"] = results.mean(axis=0).mean(axis=0)[:, 2]

    return results_, model, df

# c = [1,5,10,15,20]
# balanced = [0,1]
# dual = [0,1]
# ensemble_size = [1,2,3,4,5]
# use_pairwise = [True, False]

# results = []
# for c_, balanced_, dual_, ensemble_size_, use_pairwise_ in product(c, balanced, dual, ensemble_size, use_pairwise):
#     results_, model, df = train(c=c_, balanced=balanced_, dual=dual_, ensemble_size=ensemble_size_, use_pairwise=use_pairwise_)
#     print(results_["accuracy"], results_["f1"])
#     results.append((c_, balanced_, dual_, ensemble_size_, use_pairwise_, results_["accuracy"], results_["f1"]))



In [ ]:


def train(c=10, balanced=0, dual=1, ensemble_size=1, use_pairwise=True, use_scaler=True):
    df = pd.read_csv("tests/set4_homooligomers.csv", sep="\t")
    df = df.drop_duplicates(subset="full_sequence", keep="first")
    
    # Initialize results arrays for both target variables
    results_parallel = np.zeros((ensemble_size, 5, len(df), 2))
    results_oligo = np.zeros((ensemble_size, 5, len(df), 3))
    
    model = {}
    for j in range(0, ensemble_size):
        for i in range(0, 5): # 5 since we have 5 AF2 models
            X = np.asarray([get_af2_emb(id_, model_id=i, use_pairwise=use_pairwise) for id_ in df.index])
            
            # Combine target variables into a single array
            y_parallel = df['parallel'].values
            le = LabelEncoder()
            df['oligo_state'] = le.fit_transform(df['chains'])
            y_state = df['oligo_state'].values
            y = np.column_stack((y_parallel, y_state))
            
            cv = KFold(n_splits=5, shuffle=True)
            for k, (tr_idx, te_idx) in enumerate(cv.split(X, y)):
                X_tr, X_te = X[tr_idx], X[te_idx]
                y_tr, y_te = y[tr_idx], y[te_idx]

                if use_scaler == 1:
                    sc = StandardScaler()
                    X_tr = sc.fit_transform(X_tr)
                    X_te = sc.transform(X_te)
                    model[f"scaler_{j}_{i}_{k}"] = sc
                
                # Train and evaluate multi-output model
                clf = MultiOutputClassifier(LogisticRegression(C=c, max_iter=2000, solver='liblinear',
                                         dual=False if dual == 0 else True,
                                         class_weight='balanced' if balanced == 1 else None))
                clf.fit(X_tr, y_tr)
                proba_parallel, proba_chains = clf.predict_proba(X_te)
                results_parallel[j, i, te_idx] = proba_parallel
                results_oligo[j, i, te_idx] = proba_chains
                model[f"clf_{j}_{i}_{k}"] = clf

    # Calculate average probabilities and predicted classes for both target variables
    avg_proba_parallel = results_parallel.mean(axis=0).mean(axis=0)
    avg_proba_state = results_oligo.mean(axis=0).mean(axis=0)
    
    y_pred_bin_parallel = avg_proba_parallel.argmax(axis=1)
    y_pred_bin_state = avg_proba_state.argmax(axis=1)
    
    results_ = {}
    
    # Calculate accuracy and F1 score for parallel target variable
    results_["accuracy_parallel"] = accuracy_score(y_parallel, y_pred_bin_parallel)
    results_["f1_parallel"] = f1_score(y_parallel, y_pred_bin_parallel, average='macro')
    
    # Calculate accuracy and F1 score for chains target variable
    results_["accuracy_oligo_state"] = accuracy_score(y_state, y_pred_bin_state)
    results_["f1_oligo_state"] = f1_score(y_state, y_pred_bin_state, average='macro')

    df["y_pred_parallel"] = y_pred_bin_parallel
    df["y_pred_chains"] = y_pred_bin_state
    df["prob_dimer"] = avg_proba_state[:,0]
    df["prob_trimer"] = avg_proba_state[:,1]
    df["prob_tetramer"] = avg_proba_state[:, 2]
    df["prob_parallel"] = avg_proba_parallel[:, 1]
    df["prob_antiparallel"] = avg_proba_parallel[:, 0]
    df.to_csv('model_results.csv')

    return results_, model, df  


train()


In [ ]:
df = pd.read_csv("model/results.csv")
df

In [ ]:
from sklearn.metrics import confusion_matrix
import  seaborn as sns
confusion_matrix(df.parallel, df.y_pred_parallel)
# sns.heatmap(confusion_matrix(df.parallel, df.y_pred_parallel), annot=True, cmap="Blues", fmt="d")
confusion_matrix(df.oligo_state, df.y_pred_chains)
sns.heatmap(confusion_matrix(df.oligo_state, df.y_pred_chains), annot=True, cmap="Blues", fmt="d")

In [ ]:
df = pd.read_csv("new_model_results.csv")
le = LabelEncoder()
df['parallel_code'] = le.fit_transform(df['parallel'])
df[['pdb','parallel_code','parallel']].loc[df.pdb == '4dzo'].head(10)

In [ ]:
import glob
from src.predictor import predict_oligo_state_and_topology
import pandas as pd

# path = glob.glob('tests/data/*/')
# df = pd.DataFrame()
# for f in path:
#     df = pd.concat([df, predict_oligo_state_and_topology(f, use_pairwise=True)], axis=0)
# df.to_csv('tests/test_df.csv', index=False)

In [ ]:
tdf  = pd.DataFrame()
for f in path:
    tdf = pd.concat([tdf, predict_oligo_state_and_topology(f, use_pairwise=True)], axis=0)
tdf

In [ ]:
df  = pd.read_csv('tests/test_df.csv').reset_index(drop=True)
df

In [ ]:
from src.predictor import predict_oligo_state_and_topology
df = pd.DataFrame()

test_cases = [x for x in glob.glob('/home/nfs/rmadaj/IDUB/dc2_oligo/tmp/*/') if 'env' not in x]
pdb_annot = [x.split('/')[-2].split('_')[0] for x in glob.glob('tmp/*/*_env/')]
test_cases
for pdb, test_case in zip(pdb_annot,test_cases):
    try:
        tdf = predict_oligo_state_and_topology(test_case, use_pairwise=True)
        tdf['pdb'] = pdb
        df = pd.concat([df, tdf], axis=0)
    except:
        print(test_case, pdb)
        pass
df  = df.sort_values(by=['pdb'])
df

In [ ]:
import pandas as pd
from biopandas.pdb import PandasPdb
import sys
sys.path.append('/home/nfs/sdunin/scr/localpdb/')
from localpdb import PDB
from Bio.Data.IUPACData import protein_letters_3to1
import gzip

topdf = pd.read_pickle('/home/nfs/rmadaj/pdb_scan_nr_topology_20200717.p')
tdf = topdf.drop_duplicates(subset=['pdb']).reset_index(drop=True)
pdbs = tdf.head(2000)[['pdb']].values.flatten()

lpdb = PDB('/home/db/localpdb/', version=20210716)
input_dict = {}

for pdb in pdbs:
    entry = lpdb.entries.loc[lpdb.entries.index==pdb].pdb_fn.values[0]
    seq = ''.join(PandasPdb().read_pdb(entry).amino3to1()['residue_name'])
    if len(seq) > 300:
        continue
    if len(seq) < 30:
        continue
    if '?' in seq:
        continue
    input_dict[pdb] = seq


In [ ]:
len(input_dict)

In [ ]:
topdf

In [ ]:
# import os
# i = 0
# for key, value in input_dict.items():
#     print(key, value)
#     os.makedirs(f'tmp/{i}', exist_ok=True)
#     seq = value
#     pdb = key
    
#     with open(f'tmp/{i}/{key}_monomer.csv', 'w') as f:
#         f.write('id,sequence\n')
#         f.write(f'{pdb},{seq}\n')
        
#     with open(f'tmp/{i}.sh', 'w') as f:
#         f.write('#!/bin/bash\n')
#         f.write(f'cd ./{i}\n')
#         f.write('source /opt/miniconda3/bin/activate cf_1.5\n')
#         f.write(f'colabfold_batch {key}_monomer.csv . --num-models 5 --model-type alphafold2_multimer_v3 --num-recycle 5 --save-single-representations --save-pair-representations\n')
#     i += 1

In [ ]:
# with open(f'tmp/batch.sh', 'w') as f:
#     f.write("""#!/bin/bash
# #SBATCH -p gpu
# #SBATCH -n 4
# #SBATCH --exclude=edi0[6-8]
# #SBATCH --gres=gpu:1
# #SBATCH --mem=16GB
# #SBATCH -J dc2_bench
# #SBATCH --array 0-417

# bash $SLURM_ARRAY_TASK_ID.sh

# """)

In [ ]:
tdf = tdf.loc[tdf.pdb.isin(input_dict.keys())].reset_index(drop=True)
tdf['oligo_state'] = tdf.toplogy_len.apply(lambda x: 0 if x == 2 else 1 if x == 3 else 2)
tdf['parallel'] = tdf.topology_class.apply(lambda x: 0 if '_0' in  x else 1)
tdf = tdf.sort_values(by=['pdb'])
tdf
tempdf =  df.loc[df.pdb.isin(tdf.pdb)].sort_values(by=['pdb'])
tdf

In [ ]:
# tempdf.loc[tempdf.pdb.isin(tdf.pdb.values)]
df_compare = pd.read_pickle('/home/nfs/rmadaj/pdb_scan_nr_topology_20200717.p')
df_compare = df_compare.loc[df_compare.pdb.isin(df.pdb.values)].reset_index(drop=True).drop_duplicates(subset=['bundle_id']).reset_index(drop=True)
df_compare['oligo_state'] = df_compare.toplogy_len.apply(lambda x: 0 if x == 2 else 1 if x == 3 else 2)
df_compare['parallel'] = df_compare.topology_class.apply(lambda x: 0 if '_0' in  x else 1)


In [ ]:
df.com

In [ ]:
df_compare = df_compare.sort_values(by=['pdb'])

# for df_compare check what values of 'oligo state' per pdb match with df
for i, row in df_compare.iterrows():
    if row.pdb in df.pdb.values:
        if row.oligo_state == df.loc[df.pdb == row.pdb].y_pred_oligo.values[0]:
            print(row.pdb, row.oligo_state, df.loc[df.pdb == row.pdb].y_pred_oligo.values[0])
        else:
            print(row.pdb, row.oligo_state, df.loc[df.pdb == row.pdb].y_pred_oligo.values[0], 'WRONG')

In [ ]:
df_compare.to_csv('df_compare.csv', index=False)
df.to_csv('df_pred.csv', index=False)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm1 = confusion_matrix(df.y_pred_oligo, df_compare.oligo_state)
sns.heatmap(cm1, annot=True, cmap="Blues", fmt="d")
#accuracy
cm1.diagonal().sum()/cm1.sum()

# cm2 = confusion_matrix(df.y_pred_parallel, tdf.parallel)
# sns.heatmap(cm2, annot=True, cmap="Blues", fmt="d")

In [ ]:
ddf = pd.read_csv('tests/set4_homooligomers.csv', sep = "\t")
# calculate average length of full sequence
ddf['full_sequence_len'] = ddf.full_sequence.apply(lambda x: len(x))
ddf.full_sequence_len.mean()
ddf.full_sequence_len.median()
ddf.full_sequence_len.max()